In [1]:
import xarray as xr
import numpy as np
import copernicusmarine
import matplotlib.pyplot as plt
import matplotlib
import os
import matplotlib.gridspec as gridspec
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cmocean
from ndbc_api import NdbcApi
from datetime import datetime, timedelta
import requests, tarfile, tqdm
import pandas as pd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
import glob
from cartopy.feature import GSHHSFeature
import cartopy.io.shapereader as shpreader
import matplotlib.dates as mdates

plt.rcParams['font.family'] = 'Helvetica'  # Clean and modern
# plt.rcParams['font.family'] = 'DejaVu Sans'  # Has full font weights


In [2]:
# Set up dates and get station data
today = datetime.today()
dates = {
    'today': today.strftime('%Y-%m-%d'),
    'yesterday': (today - timedelta(days=1)).strftime('%Y-%m-%d'),
    'tomorrow': (today + timedelta(days=3)).strftime('%Y-%m-%d')
}

time_range = pd.date_range(
    start=dates['yesterday'],
    end=dates['tomorrow'],
    freq='D'
)

yday = "".join(dates["yesterday"].split("-"))
tday = "".join(dates["today"].split("-"))

In [ ]:
# Get station info
api = NdbcApi()
# station_id = '44090' #Nantucket
# station_id = '46221' #LA
station_id = 'BZBM3'; station_name = "Woods Hole, MA"
station_meta = api.station(station_id=station_id)
location = station_meta["Location"].split()
latitude = float(location[0]) * (-1 if location[1] in ['S'] else 1)
longitude = float(location[2]) * (-1 if location[3] in ['W'] else 1)

In [ ]:
bathy = xr.open_dataset("gebco_woho.nc").elevation
bathy = bathy.where(bathy < 0)

In [ ]:
# Get and process buoy data
stdmet_df = (api.get_data(station_ids=[station_id], mode='stdmet',
                         start_time=dates['yesterday'],
                         end_time=dates['tomorrow'])
             .rename_axis(index='time')
             .pipe(xr.Dataset.from_dataframe))
stdmet_df_final = stdmet_df.isel(time=~stdmet_df["WTMP"].isnull().values).isel(time=-1)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.dates as mdates
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cmocean
import numpy as np

# Global style tweaks for web
plt.rcParams.update({
    'font.family': 'DejaVu Sans',
    'axes.titlesize': 11,
    'axes.labelsize': 10,
    'xtick.labelsize': 8,
    'ytick.labelsize': 8,
    'savefig.transparent': True,
})

# Create vertically stacked figure
fig = plt.figure(figsize=(6, 8), facecolor='white', dpi=200)
gs = gridspec.GridSpec(2, 1, height_ratios=[1, 1.2], hspace=0.3)

# -------- Top: Bathymetry Map --------
ax1 = fig.add_subplot(gs[0], projection=ccrs.PlateCarree())

# Restore all spines on the map
for spine in ax1.spines.values():
    spine.set_visible(True)

# Map features
ax1.add_feature(cfeature.LAND, facecolor='darkgrey', zorder=0)
# ax1.coastlines(resolution='10m', linewidth=0.6, zorder=1)
ax1.add_feature(cfeature.BORDERS, linewidth=0.4, edgecolor='black', zorder=2)
ax1.add_feature(cfeature.STATES, linewidth=0.3, edgecolor='black', zorder=2)

# Bathymetry
contour = ax1.contourf(bathy.lon, bathy.lat, bathy,
                       levels=np.arange(-100, 0.1, 5),
                       cmap=cmocean.cm.ice, extend="both",
                       transform=ccrs.PlateCarree())
cbar = fig.colorbar(contour, ax=ax1, orientation='horizontal',
                    pad=0.05, fraction=0.05, label='Depth [m]')
cbar.ax.tick_params(labelsize=8)

# Zoom on Woods Hole
dlon, dlat = 0.2, 0.4
extent = [longitude - dlon, longitude + dlon, latitude - dlat, latitude + dlat]
ax1.set_extent(extent)

# Gridlines
gl = ax1.gridlines(draw_labels=True, linewidth=0.25, color='gray',
                   alpha=0.2, linestyle='--')
gl.top_labels = gl.right_labels = False
gl.xlabel_style = {'size': 8}
gl.ylabel_style = {'size': 8}

# Buoy marker
ax1.scatter(longitude, latitude, color='limegreen', s=80, marker='^',
            transform=ccrs.PlateCarree(), edgecolor='black', linewidth=0.5, zorder=10)

# Optional map label
ax1.text(0.01, 0.99, "Bathymetry near Woods Hole", transform=ax1.transAxes,
         ha='left', va='top', fontsize=9, weight='bold', color='black', backgroundcolor='white')

# -------- Bottom: Time Series --------
ax2 = fig.add_subplot(gs[1])
ax2.set_facecolor("#f9f9f9")

# Remove only top and right spines
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)

# Titles and location
ax2.text(0.0, 1.02, f"{station_name}\nSea Surface Temperature (°C)",
         transform=ax2.transAxes, ha='left', va='bottom', fontsize=10, weight='bold')
ax2.text(1.0, 1.02, f"Buoy: {station_id}\n{' '.join(location[0:2])}, {' '.join(location[2:4])}",
         transform=ax2.transAxes, ha='right', va='bottom', fontsize=9)

# Plot line and most recent data point
stdmet_df["WTMP"].plot(ax=ax2, color='#27ae60', linewidth=2, alpha=0.95, zorder=2)
ax2.scatter(stdmet_df_final.time, stdmet_df_final["WTMP"].values,
            color='#27ae60', edgecolor='black', s=50, zorder=5)

# Time axis formatting
ax2.xaxis.set_major_locator(mdates.AutoDateLocator())
ax2.xaxis.set_major_formatter(mdates.ConciseDateFormatter(ax2.xaxis.get_major_locator()))
plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45, ha='right')

# Labels and grid
ax2.set_xlabel('Time', fontsize=9)
ax2.set_ylabel('', fontsize=9)
ax2.grid(True, linestyle='--', linewidth=0.5, alpha=0.3, zorder=0)

# Final layout and save
fig.tight_layout()
fig.savefig("./images/temperature_plot.png", dpi=200, bbox_inches='tight')
